In [ ]:
import re
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup
import random
from random import randrange

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
list_detail_company = pd.read_csv("/content/drive/MyDrive/company_list_detail/company_list_detail.csv", index_col=[0])
list_detail_company

,id,name,alter_name,address,ceo,company_status,resgiter_place,date_start,clk
0,312617782,Công Ty TNHH Thương Mại Dịch Vụ Thiện Sinh,No name,"Số 4 Nguyễn Cửu Vân, Phường 17, Quận Bình Thạn...",Trần Thế Anh,Đang hoạt động (đã được cấp GCN ĐKT),Chi cục Thuế Quận Bình Thạnh,06/01/2014,3-754
1,309444716,Công Ty Cổ Phần Thực Phẩm An Cơ,ANCOFOOD CORP,"27 Tân Sơn Hòa, Phường 2, Quận Tân Bình, TP Hồ...",Nguyễn Thị Như Hương,Đang hoạt động (đã được cấp GCN ĐKT),Chi cục Thuế Quận Tân Bình,25/05/2011,3-754
2,312617775,Công Ty TNHH Happy Home,HAPPY HOME,"711 Lạc Long Quân, Phường 10, Quận Tân Bình, T...",Nguyễn Thanh Sang,Đang hoạt động (đã được cấp GCN ĐKT),Chi cục Thuế Quận Tân Bình,03/01/2014,3-754
3,309444709,Công Ty Cổ Phần Xây Dựng Sông Hồng,SONG HONG CONSTRUCTION COMPANY LIMITED,"Số 201C, Tòa nhà Hiệp Bình Chánh, Đường 23, Ph...",Đậu Quang Chiến,Đang hoạt động (đã được cấp GCN ĐKT),Chi cục Thuế Quận Thủ Đức,10/10/2011,3-754
4,312617768,Công Ty TNHH Một Thành Viên Thương Mại Xuất Nh...,FOUR SEASONS IMPORT EXPORT TRADE COMPANY LIMITED,"BC6 Đường 3B, Khu Dân Cư 13B, Xã Phong Phú, Hu...",Trần Thị Hiền,Đang hoạt động (đã được cấp GCN ĐKT),Chi cục Thuế Huyện Bình Chánh,07/01/2014,3-754
...,...,...,...,...,...,...,...,...,...
115,309442902,Công Ty TNHH Thương Mại Xây Dựng Và Dịch Vụ Tu...,"TUAN HUY IQ CO., LTD","215/4 Đỗ Xuân Hợp, Phường Phước Long B, Quận 9...",Nguyễn Thế Hùng,No name,Chi cục Thuế Quận 9,03/01/2012,3-754
116,312617334,Công Ty TNHH Dịch Vụ Tư Vấn Và Thương Mại Tuấn...,No name,"283 Vườn Lài, Phường Phú Thọ Hoà, Quận Tân phú...",Hà Anh Tuấn,Đang hoạt động (đã được cấp GCN ĐKT),Chi cục thuế Quận Tân phú,07/01/2014,3-754
117,309442878,Công Ty TNHH Sản Xuất Thương Mại Hoa Anh Đào,No name,"D11/1M Quách Điêu, ấp 4, Xã Vĩnh Lộc A, Huyện ...",Văn Công Sinh,Đang hoạt động (đã được cấp GCN ĐKT),Chi cục Thuế Huyện Bình Chánh,20/12/2011,3-754
118,312617327,Công Ty TNHH Dịch Vụ Thương Mại Nhật Quốc,No name,"246/11A10 Xô Viết Nghệ Tĩnh, Phường 21, Quận B...",Nguyễn Quốc Thưởng,Đang hoạt động (đã được cấp GCN ĐKT),Chi cục Thuế Quận Bình Thạnh,06/01/2014,3-754


In [ ]:
#function get company main atribute
def get_company(url):
  company_df = pd.DataFrame(columns=["id","name","address","ceo"])
  company_fail_link = []

  r = requests.get(url , headers = {'User-agent': 'your bot 0.1'})
  if r.status_code != 200:
    print("fail error to request" + str(r.status_code))
    return
  else:
    soup = BeautifulSoup(r.content, 'html.parser')
    my_table = soup.find("div",class_="m-panel")
    my_company = my_table.find_all("div", class_="company-item")

    for company in my_company:
      name = company.find("a").text
      group_id = re.search(r"Mã số thuế: (\d+)", str(company))
      group_ceo = re.search(r"Đại diện pháp luật: (.+)<", str(company))
      group_adds = re.search(r"Địa chỉ: (.+)<", str(company))
      header = company.find("a")
      group_link = re.search(r"(https:\/\/infodoanhnghiep.com\/thong-tin\/.+html)", str(header))
      link = group_link.group(1) if group_link else "No name"
      #print(link)
      id = group_id.group(1) if group_id else "No name"
      address = group_adds.group(1) if group_adds else "No name"
      ceo = group_ceo.group(1) if group_ceo else "No name"

      new_row = {'id':id , 'name':name, 'address':address, 'ceo':ceo}
      #append row to the dataframe
      company_df = company_df.append(new_row, ignore_index=True)
    return company_df 

In [ ]:
def scraper_web(page_start,page_end):
  i = page_start
  city_df = pd.read_csv("/content/drive/MyDrive/city_list/city_list.csv", index_col=[0])
  
  company_df = pd.DataFrame(columns=["id","name","address","ceo"])
  page = "https://infodoanhnghiep.com/"

  while i<page_end + 1:
    if i%100 == 1:
      if company_df.empty:
        company_df = pd.DataFrame(columns=["id","name","address","ceo"])
      else:
        print("save data page "+str(i-1))
        company_df.to_csv("/content/drive/MyDrive/company_list/company_list_"+ str(i-1))
        company_df = []
        company_df = pd.DataFrame(columns=["id","name","address","ceo"])
        
    city_df = city_df.sample(frac=1).reset_index(drop=True)
    k = 0
    while k < city_df.shape[0]:
      #print(city_df.iloc[k,1])
      if i<= int(city_df.iloc[k,1]):
        url = page + str(city_df.iloc[k,0]) + "/trang-" + str(i)
        print(url)
        new_df = get_company(url)
        company_df = pd.concat([company_df, new_df], ignore_index=True)
        k = k + 1
      else:
        random_page = randrange(int(city_df.iloc[k,1]))
        url = page + str(city_df.iloc[k,0]) + "/trang-" + str(random_page)
        if random_page % 5 == 0:
          r = requests.get(url , headers = {'User-agent': 'your bot 0.1'})
        print("fake request "+url)
        time.sleep(0.2)
        k = k + 1
    i = i+1
    time.sleep(0.5)
  return company_df

In [ ]:
def scraper_web_high_page(page_start,page_end):
  i = page_start
  city_df = pd.read_csv("/content/drive/MyDrive/city_list/city_list.csv", index_col=[0])
  city_df = city_df.iloc[0:20, :]
  company_df = pd.DataFrame(columns=["id","name","address","ceo"])
  page = "https://infodoanhnghiep.com/"

  while i<page_end + 1:
    if i%100 == 1:
      if company_df.empty:
        company_df = pd.DataFrame(columns=["id","name","address","ceo"])
      else:
        print("save data page "+str(i-1))
        company_df.to_csv("/content/drive/MyDrive/company_list/company_list_"+ str(i-1))
        company_df = []
        company_df = pd.DataFrame(columns=["id","name","address","ceo"])
        
    city_df = city_df.sample(frac=1).reset_index(drop=True)
    k = 0
    while k < city_df.shape[0]:
      #print(city_df.iloc[k,1])
      if i<= int(city_df.iloc[k,1]):
        url = page + str(city_df.iloc[k,0]) + "/trang-" + str(i)
        print(url)
        new_df = get_company(url)
        company_df = pd.concat([company_df, new_df], ignore_index=True)
        k = k + 1
      else:
        random_page = randrange(int(city_df.iloc[k,1]))
        url = page + str(city_df.iloc[k,0]) + "/trang-" + str(random_page)
        r = requests.get(url , headers = {'User-agent': 'your bot 0.1'})
        print("fake request "+url)
        #time.sleep(0.2)
        k = k + 1
    i = i+1
    time.sleep(0.5)
  return company_df

In [ ]:
get_company("https://infodoanhnghiep.com/Cao-Bang/trang-6")

,id,name,address,ceo
0,4800927859,ĐỒN BIÊN PHÒNG TỔNG CỌT,"Lũng Gioỏng, Xã Tổng Cọt, Huyện Hà Quảng, Cao ...",Lục Sơn Thuỷ
1,4800927721,CÔNG TY TNHH THƯƠNG MẠI VÀ XÂY DỰNG 2/9,"Xóm Bản Séng, Thị trấn Nước Hai, Huyện Hoà An,...",Quách Văn Nghị
2,4800927739,CÔNG TY TNHH MTV RƯỢU CỔ TRUYỀN CAO BẰNG,"Tổ 1,, Phường Sông Hiến, Thành phố Cao Bằng, C...",Lý Đạo Huy
3,4800927619,CÔNG TY TNHH KHOA HỌC VÀ CÔNG NGHỆ KỸ THUẬT HNQ,"Xóm Phia Bó,, Xã Phong Châu, Huyện Trùng Khánh...",Nông Đình Nghiêm
4,4800927489,HỢP TÁC XÃ MINH NHẬT 68,"Xóm Minh Khai, Xã Đức Long, Huyện Hoà An, Cao ...",Trịnh Thế Luân
5,4800927104,ĐỒN BIÊN PHÒNG LŨNG NẶM,"Cả Giỏng, Xã Lũng Nặm, Huyện Hà Quảng, Cao Bằng",Nguyễn Trung Kiên
6,4800926929,CÔNG TY TNHH MTV ĐẦU TƯ VÀ XÂY DỰNG TỔNG HỢP B...,"Số nhà 60, tổ 1 Đường Lê Lợi,, Phường Sông Bằn...",Vũ Ngọc Sơn
7,4800926799,CÔNG TY TNHH NÔNG HƯNG 381,"Tổ 4,, Phường Hoà Chung, Thành phố Cao Bằng, C...",Nông Văn Hưng
8,4800926728,CÔNG TY TNHH MỘT THÀNH VIÊN EAD VIỆT NAM,"Tổ 6,, Phường Ngọc Xuân, Thành phố Cao Bằng, C...",Nguyễn Mạnh Quân
9,4800926460,CÔNG TY TNHH XÂY DỰNG THIÊN TRƯỜNG NĐ,"Tổ 12,, Phường Đề Thám, Thành phố Cao Bằng, Ca...",Trần Quốc Huy


In [ ]:
company_chunk = scraper_web_high_page(4501,5000)
company_chunk.to_csv("/content/drive/MyDrive/company_list/company_list_5000")

fake request https://infodoanhnghiep.com/Bac-Lieu/trang-82
fake request https://infodoanhnghiep.com/Binh-Thuan/trang-396
fake request https://infodoanhnghiep.com/Can-Tho/trang-74
fake request https://infodoanhnghiep.com/Bac-Kan/trang-129
fake request https://infodoanhnghiep.com/Ca-Mau/trang-240
fake request https://infodoanhnghiep.com/Bac-Giang/trang-447
https://infodoanhnghiep.com/Ha-Noi/trang-4501
fail error to request403
fake request https://infodoanhnghiep.com/Cao-Bang/trang-91
fake request https://infodoanhnghiep.com/Binh-Dinh/trang-128
https://infodoanhnghiep.com/TP-Ho-Chi-Minh/trang-4501
fail error to request403
fake request https://infodoanhnghiep.com/Binh-Phuoc/trang-596


KeyboardInterrupt: ignored

In [ ]:
company_chunk.to_csv("/content/drive/MyDrive/company_list/company_list_2000")

In [ ]:
#group data company
import pandas as pd
import glob
import os

path = r'/content/drive/MyDrive/company_list/' # use your path
all_files = os.listdir(path)
all_files.remove(".ipynb_checkpoints")
print(all_files)
li = []

for filename in all_files:
    print(filename)
    df = pd.read_csv(path+filename, index_col=0, header=0)
    li.append(df)

group_df = pd.concat(li, axis=0, ignore_index=True)
group_df.drop_duplicates(keep= False, inplace=True, ignore_index=True)
group_df

['company_list_100', 'company_list_200', 'company_list_300', 'company_list_400', 'company_list_500', 'company_list_600', 'company_list_700', 'company_list_800', 'company_list_900', 'company_list_1000', 'company_list_1500', 'company_list_1600', 'company_list_1700', 'company_list_1800', 'company_list_1900', 'company_list_2000', 'company_list_2100', 'company_list_2200', 'company_list_2300', 'company_list_2400', 'company_list_2500', 'company_list_2600', 'company_list_2700', 'company_list_2800', 'company_list_2900', 'company_list_3000', 'company_list_3100', 'company_list_3200', 'company_list_3300', 'company_list_3400', 'company_list_3500', 'company_list_3600', 'company_list_3700', 'company_list_3800', 'company_list_3900', 'company_list_4000', 'company_list_4100', 'company_list_4200', 'company_list_4300', 'company_list_4400', 'company_list_4500']
company_list_100
company_list_200
company_list_300
company_list_400
company_list_500
company_list_600
company_list_700
company_list_800
company_lis

,id,name,address,ceo
0,4001252166,CÔNG TY TNHH XÂY DỰNG TÍN XUÂN QUẢNG NAM,"Tổ 3, Khối Phố Phú Sơn, Phường An Phú, Thành p...",Lê Văn Tín
1,4001252159,CÔNG TY TNHH TMDV DOANH THỊNH,"Đường Nguyễn Thành Hãn, Khu phố chợ Nam Phước,...",Đoàn Thị Thùy Nhung
2,4001252127,CÔNG TY TNHH THƯƠNG MẠI VÀ DỊCH VỤ NHẬT BẢO NAM,"Lô C19, KDC Tây Yên mở rộng, Xã Tam Đàn, Huyện...",Nguyễn Ngọc Hải
3,4001252134,CÔNG TY TNHH THÊU VI TÍNH LONG EMB,"Tổ 4, Thôn Khánh Tân, Xã Tam Dân, Huyện Phú Ni...",Phạm Văn Lợi
4,4001252102,CÔNG TY TNHH ĐẦU TƯ VÀ PHÁT TRIỂN CÔNG NGHỆ VẬ...,"56 Lê Lợi, Phường An Mỹ, Thành phố Tam Kỳ, Quả...",Phan Thanh Hoàng
...,...,...,...,...
1049031,314445345,CÔNG TY TNHH THỰC PHẨM SÀI GÒN THẢO MINH,"Số 368, 368A Đường Dương Quảng Hàm, Phường 5, ...",Nguyễn Thị Bích Ngà
1049032,314445313,CÔNG TY TNHH THƯƠNG MẠI XUẤT NHẬP KHẨU PHÁT TR...,"968 Bàu Tre 2, Xã Tân An Hội, Huyện Củ Chi, TP...",Lê Hồng Thắm
1049033,314445296,CÔNG TY TNHH XUẤT NHẬP KHẨU HẠO NAM KOREA,"68 ấp Tiền, Xã Tân Thông Hội, Huyện Củ Chi, TP...",Nguyễn Văn Khởi
1049034,314445289,CÔNG TY TNHH THƯƠNG MẠI DỊCH VỤ W3WEB,"14/13/41 Thân Nhân Trung, Phường 13, Quận Tân ...",Đồng Như Hải


In [ ]:
group_df

,id,name,address,ceo
0,4001252166,CÔNG TY TNHH XÂY DỰNG TÍN XUÂN QUẢNG NAM,"Tổ 3, Khối Phố Phú Sơn, Phường An Phú, Thành p...",Lê Văn Tín
1,4001252159,CÔNG TY TNHH TMDV DOANH THỊNH,"Đường Nguyễn Thành Hãn, Khu phố chợ Nam Phước,...",Đoàn Thị Thùy Nhung
2,4001252127,CÔNG TY TNHH THƯƠNG MẠI VÀ DỊCH VỤ NHẬT BẢO NAM,"Lô C19, KDC Tây Yên mở rộng, Xã Tam Đàn, Huyện...",Nguyễn Ngọc Hải
3,4001252134,CÔNG TY TNHH THÊU VI TÍNH LONG EMB,"Tổ 4, Thôn Khánh Tân, Xã Tam Dân, Huyện Phú Ni...",Phạm Văn Lợi
4,4001252102,CÔNG TY TNHH ĐẦU TƯ VÀ PHÁT TRIỂN CÔNG NGHỆ VẬ...,"56 Lê Lợi, Phường An Mỹ, Thành phố Tam Kỳ, Quả...",Phan Thanh Hoàng
...,...,...,...,...
1029031,314563726,CÔNG TY TNHH S-TRADING,"Tầng 15, Tòa nhà Petronas, số 235 Nguyễn Văn C...",Phùng Đắc Quang
1029032,314563701,CÔNG TY TNHH DỊCH VỤ - TƯ VẤN - DU LỊCH - XUẤT...,"155 Ngô Thị Thu Minh, Phường 2, Quận Tân Bình,...",Phùng Thị Thảo
1029033,314563684,CÔNG TY TNHH THƯƠNG MẠI XUẤT NHẬP KHẨU TÍN HỘI,"76 Đường 24A, Phường Bình Trị Đông B, Quận Bìn...",Lâm Yến Linh
1029034,314563363,CÔNG TY TNHH SẮC MÀU PLASTIC,"Số 27 Đường Số 1, ấp ông Nhiêu, Phường Long Tr...",Lê Nguyễn Thanh Tuấn


In [ ]:
company_df = pd.read_csv("/content/drive/MyDrive/company_list/company_list_100", index_col=[0])
company_df

,id,name,address,ceo
0,3703062661,CÔNG TY TNHH THƯƠNG MẠI DỊCH VỤ CÔNG NGHIỆP VĨ...,"Thửa đất số 1462, Tờ bản đồ số 10, Đường D8, K...",Trần Nhịt Và
1,3703062679,DOANH NGHIỆP TN NHÀ TRỌ CHO THUÊ LƯU TRÚ TÁM BỜ,"Thửa đất số 911, 912, Tờ bản đồ số 6, Đường Câ...",Nguyễn Thị Phấn
2,3703062615,CÔNG TY TNHH XÂY LẮP HOÀNG KIM,"15/2, Tổ 2, Khu phố Khánh Long, Đường Tân Phướ...",Nguyễn Quốc Nam
3,3703062654,CÔNG TY TNHH THƯƠNG MẠI DỊCH VỤ TRANGPAP,"Lô B7, ô 19 Khu nhà ở thương mại, Đường Lý Thư...",Ngô Huyền Trang
4,3703062703,CÔNG TY TNHH XUẤT NHẬP KHẨU MỘC AN NHIÊN,"Số 10, Đường 02, Khu nhà ở thương mại Hoàng Na...",Lê Phước Hoài Chi
...,...,...,...,...
127255,3502409514,CÔNG TY TNHH MTV VẬN TẢI KÍNH HIẾU,"0164 Vĩnh Bình, Xã Bình Giã, Huyện Châu Đức, B...",Nguyễn Thị Thúy Hằng
127256,3502409384,CÔNG TY CỔ PHẦN ĐẦU TƯ DỊCH VỤ XÂY DỰNG PHÚ KHANG,"Số 2/52 Đống Đa, Phường Thắng Nhất, Thành Phố ...",Phạm Đình Tiệp
127257,3502409497,CÔNG TY TNHH ĐỨC MAI CÔN ĐẢO,"Khu 6, Thị trấn Côn Đảo, Huyện Côn Đảo, Bà Rịa...",Phú Thị Tuyết Mai
127258,3502409465,CÔNG TY TNHH YOUNGWIRE VT,"Lô 29, Đường Đ.10, Khu công nghiệp Châu Đức, T...",Lee Hwang Jae


In [ ]:
#function get company detail access
#function get company atribute

def get_company_detail(url):
  company_detail_df = pd.DataFrame(columns=["id","name","alter_name","address","ceo", "company_status" , "resgiter_place", "date_start", "clk"])

  r = requests.get(url , headers = {'User-agent': 'your bot 0.1'})
  if r.status_code != 200:
    print("fail to request " + str(r.status_code))
    return
  else:
    #print(r.status_code)
    soup = BeautifulSoup(r.content, 'html.parser')
    my_table = soup.find("div",class_="m-panel")
    my_company = my_table.find_all("div", class_="company-item")

    for company in my_company:
      name = company.find("a").text
      group_id = re.search(r"Mã số thuế: (\d+)", str(company))
      group_ceo = re.search(r"Đại diện pháp luật: (.+)<", str(company))
      group_adds = re.search(r"Địa chỉ: (.+)<", str(company))
      header = company.find("a")
      group_link = re.search(r"(https:\/\/infodoanhnghiep.com\/thong-tin\/.+html)", str(header))
      company_link = group_link.group(1) if group_link else "No name"
      print(company_link)
      id = group_id.group(1) if group_id else "No name"
      address = group_adds.group(1) if group_adds else "No name"
      ceo = group_ceo.group(1) if group_ceo else "No name"


      #request detail company page
      r = requests.get(company_link , headers = {'User-agent': 'your bot 0.1'})
      soup = BeautifulSoup(r.content, 'html.parser')
      my_description = soup.find("div",class_="description", itemprop="description")
      my_p = my_description.find_all("p")
      description = my_p[1].text if my_p else "No name"
      my_detail_table = soup.find("div",class_="responsive-table responsive-table-2cols responsive-table-collapse")
      alter_name = my_detail_table.find("div", class_="responsive-table-cell", itemprop="alternateName")
      alter_name = alter_name.text if alter_name else "No name"
      group_company_status = re.search(r"Tình trạng hoạt động:.+>(.+\))<", str(my_detail_table))
      company_status = group_company_status.group(1) if group_company_status else "No name"
      group_resgiter_place = re.search(r"Nơi đăng ký quản lý.+\n.+>(.+)<", str(my_detail_table))
      resgiter_place = group_resgiter_place.group(1) if group_resgiter_place else "No name"
      group_date_start = re.search(r"Ngày cấp giấy phép:.+>(\d+\/\d+\/\d+)",str(my_detail_table))
      date_start = group_date_start.group(1) if group_date_start else "No name"
      group_clk = re.search(r"Cấp Chương Loại Khoản.+>(\d+-\d+)",str(my_detail_table))
      clk = group_clk.group(1) if group_clk else "No name"

      new_row = {"id":id,"name":name,"alter_name":alter_name,"address":address,"ceo":ceo, "company_status":company_status , "resgiter_place":resgiter_place, "date_start":date_start, "clk":clk}
      #append row to the dataframe
      company_detail_df = company_detail_df.append(new_row, ignore_index=True)
  return company_detail_df

In [ ]:
def scraper_web_detail(page_start,page_end):
  i = page_start
  city_df = pd.read_csv("/content/drive/MyDrive/city_list/city_list.csv", index_col=[0])
  #city = ['Kien-Giang','Tien-Giang','Ha-Nam','Ha-Noi','Nghe-An','Ninh-Binh','Thanh-Hoa','Hai-Duong','Ha-Tinh','Thai-Binh','Da-Nang','Hung-Yen','Hai-Phong']
  company_detail_df = pd.DataFrame(columns=["id","name","alter_name","address","ceo", "company_status" , "resgiter_place", "date_start", "clk"])
  while i<page_end + 1:
    city_df = city_df.sample(frac=1).reset_index(drop=True)
    #random.shuffle(list_city)
    k = 0
    while k < city_df.shape[0]:
      #print(city_df.iloc[k,1])
      if i<= int(city_df.iloc[k,1]):
        page = "https://infodoanhnghiep.com/"
        url = page + str(city_df.iloc[k,0]) + "/trang-" + str(i)
        print(url)
        new_df = get_company_detail(url)
        company_detail_df = pd.concat([company_detail_df, new_df], ignore_index=True)
        k = k + 1
      else:
        page = "https://infodoanhnghiep.com/"
        url = page + str(city_df.iloc[k,0]) + "/trang-" + str(randrange(int(city_df.iloc[k,1])))
        r = requests.get(url , headers = {'User-agent': 'your bot 0.1'})
        print("fake request "+url)
        time.sleep(0.25)
        k = k + 1
    i = i+1
    time.sleep(0.25)
  company_detail_df.to_csv("/content/drive/MyDrive/company_list_detail/company_list_detail.csv")
  return company_detail_df

In [ ]:
city_df = pd.read_csv("/content/drive/MyDrive/city_list/city_list.csv", index_col=[0])
city_df.head(60)

,city,count
0,Ha-Noi,15646
1,TP-Ho-Chi-Minh,25104
2,Can-Tho,1055
3,Da-Nang,2392
4,Hai-Phong,2431
5,An-Giang,728
6,Ba-Ria-Vung-Tau,1201
7,Bac-Giang,631
8,Bac-Kan,155
9,Bac-Lieu,299


In [ ]:
scraper_web_detail(100,110)

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
https://infodoanhnghiep.com/thong-tin/Cong-ty-co-phan-280-28741.html
https://infodoanhnghiep.com/thong-tin/DNTN-Thuong-mai-va-dich-vu-Hai-Lan-28734.html
https://infodoanhnghiep.com/Ha-Giang/trang-107
https://infodoanhnghiep.com/thong-tin/UBND-Xa-Phu-Nam-66637.html
https://infodoanhnghiep.com/thong-tin/Toa-an-nhan-dan-huyen-Vi-Xuyen-66605.html
https://infodoanhnghiep.com/thong-tin/Truong-trung-hoc-co-so-Ngoc-Linh-66595.html
https://infodoanhnghiep.com/thong-tin/So-tai-nguyen-va-moi-truong-Ha-Giang-66570.html
https://infodoanhnghiep.com/thong-tin/Cong-ty-trach-nhiem-huu-han-mot-thanh-vien-Ngoc-Trung-66563.html
https://infodoanhnghiep.com/thong-tin/Hop-tac-xa-Huy-Hoang-66556.html
https://infodoanhnghiep.com/thong-tin/Trung-Tam-Giao-Duc-Thuong-xuyen-66450.html
https://infodoanhnghiep.com/thong-tin/Phong-Thanh-Tra-Nha-Nuoc-66436.html
https://infodoanhnghiep.com/thong-tin/Phong-Van-Hoa-Va-The-Thao-Bac-Me-66387.html
https://infodoanhngh

,id,name,alter_name,address,ceo,company_status,resgiter_place,date_start,clk
0,4900239006,Công Ty TNHH Sản Xuất Và Thương Mại An Lộc,ANLOC PRODUCTION AND TRADE COMPANY LIMITED,"Số 14, phố Hoà Bình, Thị trấn Lộc Bình, Huyện ...",Lưu Thuý Châm,Đang hoạt động (đã được cấp GCN ĐKT),Chi cục Thuế Huyện Lộc Bình,17/10/2011,3-754
1,4900238796,Công Ty Cổ Phần Sản Xuất - Thương Mại Tranh Th...,No name,"Lô M7, cụm Công nghiệp địa phương số 2, Xã Hợp...",Hoàng Thị Bình,Đang hoạt động (đã được cấp GCN ĐKT),Chi cục Thuế Huyện Cao Lộc,09/08/2011,3-754
2,4900238764,CÔNG TY CỔ PHẦN ĐẦU TƯ XÂY DỰNG VÀ PHÁT TRIỂN ...,"VIETBAC ., JSC","Số 78, đường Lê Lợi, Phường Vĩnh Trại, Thành p...",Hoàng Thế Cường,Đang hoạt động (đã được cấp GCN ĐKT),Cục Thuế Tỉnh Lạng Sơn,13/09/2011,555-161
3,4900238757,Công Ty Cổ phần Thương mại và Đầu tư xây dựng ...,No name,"Số 156, Đường Bến Bắc, Phường Tam Thanh, Thành...",Vũ Thanh Danh,Đang hoạt động (đã được cấp GCN ĐKT),Cục Thuế Tỉnh Lạng Sơn,08/02/2005,2-554
4,4900238732,Công ty cổ phần xây dựng - Thương mại Lạng Sơn,No name,"Số 32, Đường Chu Văn An, Phường Vĩnh Trại, Thà...",Phạm Dụng,No name,Chi cục Thuế Thành phố Lạng Sơn,07/02/2006,3-754
...,...,...,...,...,...,...,...,...,...
13855,3603801333,CÔNG TY TNHH XUẤT NHẬP KHẨU KINOKO LONG KHÁNH,LONG KHANH KINOKO IMPORT - EXPORT COPMPANY LIM...,"Số 36, Tổ 1, đường Nguyễn Văn Bé, Phường Xuân ...",Dương Thị Thu Huệ,Đang hoạt động (đã được cấp GCN ĐKT),Cục Thuế Tỉnh Đồng Nai,31/03/2021,555-011
13856,3603801319,CÔNG TY TNHH BÁCH HÓA TỔNG HỢP AN THỊNH PHÁT,No name,"Tổ 7, ấp Bàu Trâm, Xã Bàu Trâm, Thành phố Long...",Phạm Đình Hậu,Đang hoạt động (đã được cấp GCN ĐKT),Chi cục Thuế khu vực Long Khánh - Cẩm Mỹ,31/03/2021,754-194
13857,3603801284,CÔNG TY TNHH XÂY DỰNG TUẤN ANH PHÁT,"TUAN ANH PHAT CONSTRUCTION CO., LTD","80/62 Trần Văn Xã, tổ 3, KP 5, Phường Trảng Dà...",Hồ Ngọc Tuấn,Đang hoạt động (đã được cấp GCN ĐKT),Chi cục Thuế khu vực Biên Hòa - Vĩnh Cửu,31/03/2021,755-161
13858,3603801012,CÔNG TY TNHH NÔNG NGHIỆP NÔNG GIA KHÁNH,CTY TNHH NN NôNG GIA KHáNH,"Tổ 15B, đường Hàm Nghi, KP. Ruộng Hời, Phường ...",Nguyễn Thành Luân,Đang hoạt động (đã được cấp GCN ĐKT),Chi cục Thuế khu vực Long Khánh - Cẩm Mỹ,30/03/2021,754-194


In [ ]:
scraper_web_detail(10000,10005)

fake request https://infodoanhnghiep.com/Da-Nang/trang-284
fake request https://infodoanhnghiep.com/Hai-Phong/trang-1694
fake request https://infodoanhnghiep.com/Tien-Giang/trang-572
fake request https://infodoanhnghiep.com/Long-An/trang-1062
fake request https://infodoanhnghiep.com/Khanh-Hoa/trang-1064
fake request https://infodoanhnghiep.com/Thai-Nguyen/trang-537
fake request https://infodoanhnghiep.com/Quang-Binh/trang-68
fake request https://infodoanhnghiep.com/Ba-Ria-Vung-Tau/trang-230
fake request https://infodoanhnghiep.com/Bac-Lieu/trang-143
fake request https://infodoanhnghiep.com/Lam-Dong/trang-725
fake request https://infodoanhnghiep.com/Thanh-Hoa/trang-991
fake request https://infodoanhnghiep.com/Hoa-Binh/trang-248
fake request https://infodoanhnghiep.com/Quang-Ninh/trang-4
fake request https://infodoanhnghiep.com/Tay-Ninh/trang-301
fake request https://infodoanhnghiep.com/Ben-Tre/trang-233
fake request https://infodoanhnghiep.com/Thai-Binh/trang-339
fake request https://in

,id,name,alter_name,address,ceo,company_status,resgiter_place,date_start,clk
0,0313360681,Công Ty TNHH Thương Mại Dịch Vụ Ibizone,No name,"333 Phan Xích Long, Phường 01, Quận Phú Nhuận,...",Nguyễn Thái Giang,Đang hoạt động (đã được cấp GCN ĐKT),Chi cục Thuế Quận Phú Nhuận,15/07/2015,3-754
1,0313360674,Công Ty TNHH Nidala,NIDALA COMPANY LIMITED,"Tầng 2 Tòa nhà Ngọc Đông Dương, 76 Cách Mạng T...",Lê Tuấn Hùng,Đang hoạt động (đã được cấp GCN ĐKT),Chi cục Thuế Quận 3,16/07/2015,3-754
2,0313477182,Công Ty TNHH Camera Gia Nguyễn,GIA NGUYEN CAMERA COMPANY LIMITED,"39 Lê Văn Chí, Khu phố 3, Phường Linh Trung, Q...",Nguyễn Trần Ngọc Thạch,Đang hoạt động (đã được cấp GCN ĐKT),Chi cục Thuế Quận Thủ Đức,05/10/2015,3-754
3,0313360593,Công Ty TNHH Tng Homes,"TNG HOMES CO.,LTD","R3, 85 Hưng Gia 4, Phú Mỹ Hưng, Phường Tân Pho...",Đoàn Ngọc Nga,Đang hoạt động (đã được cấp GCN ĐKT),Chi cục Thuế Quận 7,13/07/2015,3-754
4,0313477175,Công Ty TNHH Vật Liệu Xây Dựng Bảo Châu,No name,"53 Đường 12, Khu phố 5, Phường Hiệp Bình Chánh...",Nguyễn Thị Thanh Hương,Đang hoạt động (đã được cấp GCN ĐKT),Chi cục Thuế Quận Thủ Đức,05/10/2015,3-754
...,...,...,...,...,...,...,...,...,...
235,0100112606,Công ty khu vui chơi giải trí Việt Nam,AMUSEMENT WORLD OF VIETNAM INC,"Công viên Lê nin,, Quận Hai Bà Trưng, Hà Nội",Tushifumi Matsuzawa,No name,Cục Thuế Thành phố Hà Nội,25/04/1998,1-152
236,0100112589,Tổng Công ty Vàng bạc đá quý Việt nam,VIETGOLDGEM,"23 B Quang Trung,, Quận Hoàn Kiếm, Hà Nội",Vũ Dương Thanh,Đang hoạt động (đã được cấp GCN ĐKT),Cục Thuế Thành phố Hà Nội,26/04/1998,1-120
237,0100112444,"Ngân Hàng Citibank, Na., - Chi Nhánh Hà Nội","CITIBANK, NA HA NOI BRANCH","Số 17, phố Ngô Quyền, Phường Lý Thái Tổ, Quận ...",Natalia Olegovna Ansell,Đang hoạt động (đã được cấp GCN ĐKT),Cục Thuế Thành phố Hà Nội,20/04/1998,1-151
238,0100112081,Công ty liên doanh VSV,.,"104 Phố hoà mã,, Quận Hai Bà Trưng, Hà Nội",Aeimporn Punyasai,No name,Cục Thuế Thành phố Hà Nội,20/04/1998,1-152


In [ ]:
def scraper_web_detail(page_start,page_end):
  i = page_start
  city_df = pd.read_csv("/content/drive/MyDrive/city_list/city_list.csv", index_col=[0])
  #city = ['Kien-Giang','Tien-Giang','Ha-Nam','Ha-Noi','Nghe-An','Ninh-Binh','Thanh-Hoa','Hai-Duong','Ha-Tinh','Thai-Binh','Da-Nang','Hung-Yen','Hai-Phong']
  company_detail_df = pd.DataFrame(columns=["id","name","alter_name","address","ceo", "company_status" , "resgiter_place", "date_start", "clk"])
  while i<page_end + 1:
    city_df = city_df.sample(frac=1).reset_index(drop=True)
    #random.shuffle(list_city)
    k = 0
    while k < city_df.shape[0]:
      #print(city_df.iloc[k,1])
      if i<= int(city_df.iloc[k,1]):
        page = "https://infodoanhnghiep.com/"
        url = page + str(city_df.iloc[k,0]) + "/trang-" + str(i)
        print(url)
        new_df = get_company_detail(url)
        company_detail_df = pd.concat([company_detail_df, new_df], ignore_index=True)
        k = k + 1
      else:
        #time.sleep(0.01)
        k = k + 1
    i = i+1
    time.sleep(0.25)
  company_detail_df.to_csv("/content/drive/MyDrive/company_list_detail/company_list_detail.csv")
  return company_detail_df

In [ ]:
scraper_web_detail(20009,20019)

https://infodoanhnghiep.com/TP-Ho-Chi-Minh/trang-20009
fail to request 403
https://infodoanhnghiep.com/TP-Ho-Chi-Minh/trang-20010
fail to request 403
https://infodoanhnghiep.com/TP-Ho-Chi-Minh/trang-20011
fail to request 403


KeyboardInterrupt: ignored